## Εργαστήριο 7

Όνομα: Μάριος Παπαχρήστου

ΑΜ: 03115101

email : papachristoumarios@gmail.conm

### Εργοδικός Μέσος (Άσκηση 103)

Στην άσκηση αυτή γίνεται προσομοίωση της αλυσίδας της  ́*Ασκησης 80* για $p = 1/2$.
Υπενθυμίζεται ότι αυτή η αλυσίδα στον χώρο καταστάσεων $ \mathbb{N} \cup \{0\}$ 
είναι μη υποβιβάσιμη, γνησίως επαναληπτική, με αναλλοίωτη κατανομή $\pi$ που δίνεται από την

$$ \pi(k) = \frac{1}{2^{k+1}}, \quad k = 0, 1, 2, 3, ... $$

Ο κώδικας στο παρακάτω κελί προσομοιώνει τα πρώτα $N = 10^4$ βήματα της αλυσίδας 
και επιστρέφει τον εργοδικό μέσο

$$
\frac{X_1 + X_2 + ... + X_N}{N}
$$

Από το εργοδικό Θεώρημα η παραπάνω ποσότητα συγκλίνει καθώς $N \to \infty$ με πιθανότητα 1 στο 

$$ \sum_{k=0}^{\infty} k \pi(k) $$

οπότε ο κώδικας υπολογίζει αριθμητικά την παραπάνω ποσότητα με τη μέθοδο Markov Chain Monte
Carlo (MCMC).


In [1]:
import numpy as np


def random_walk_next(state):
    if np.random.uniform() < 0.5:
        return 0
    return state + 1

def ergodic_sum(N = 10**4):
    running_state = 0
    sum_result = 0

    for i in range(N):
        running_state = random_walk_next(running_state)
        sum_result += running_state

    return sum_result / N 

    
### Ergodic Limit Theorem
Se = ergodic_sum()
print("The simulated ergodic average [X1+X2+X3+...+XN]/N is: %.4f" % Se)

The simulated ergodic average [X1+X2+X3+...+XN]/N is: 0.9526


### Παραδοτέο 1


In [2]:
#1 
inf_sum  = lambda N = 1000 : sum([k / 2**(k+1) for k in range(N)])
S = inf_sum()
print('The sum using 1000 terms is {}'.format(S))
r = abs(Se - S) / S
print('Relative error is {}'.format(r))

#2
sums = np.array([ergodic_sum() for i in range(100)])
print('Variance of sums is ', np.var(sums, ddof = 1))

#3
sums = np.array([ergodic_sum(N = 2 * 10**4) for i in range(100)])
print('Variance of sums is ', np.var(sums, ddof = 1))


The sum using 1000 terms is 0.9999999999999999
Relative error is 0.047399999999999894
Variance of sums is  0.000679110580808
Variance of sums is  0.000291085396717


1. Τα αθροίσματα υπολογίζονται με τις παραπάνω ρουτίνες
2. Η διακύμανση των αθροισμάτων είναι 0.0006
3. Από τις ιδιότητες της διασποράς προκύπτει ότι $\sigma'^2 = \sigma^2 / 4$ από το οποίο λαμβάνουμε $N' = 2N$

4. Έστω $f : \mathbb N \to \mathbb [-1, 1]$ με $f(x) = 2 \cos (x + \cos x)$ και $|f(x)| \le 2$. Τότε από το εργοδικό θεώρημα ΙΙ έχουμε ότι το ζητούμενο άθροισμα δίνεται από τον εργοδικό μέσο για $N \to \infty$ $$\frac 1 N \sum_{k = 1}^N f(X_k) $$

In [3]:
def ergodic_sum_2(N = 10**4):
    running_state = 0
    sum_result = 0

    for i in range(N):
        running_state = random_walk_next(running_state)
        sum_result += np.cos(running_state + np.cos(running_state)) * 2 

    return sum_result / N 

S2 = ergodic_sum_2()
print('Sum of Σ cos(k + cos(k)) / 2^k) = ', S2)

Sum of Σ cos(k + cos(k)) / 2^k) =  0.47634478851


## Παραδοτέο 2

Παρατηρούμε ότι η εκτίμηση με εργοδικό μέσο δίνει μικρότερα σφάλματα. 

In [46]:
import numpy as np
import random as r
from math import gamma, pi 

def Vol1(d):
    x = d/2
    return pi ** x / gamma(x + 1)


def mcmc_volume(dmax = 10, N = 100, delta = 1.0):
    samples = 1000 
    D = {}
    D[1] = 2 
    for d in range(1, dmax + 1):
        pts = []
        for _ in range(samples):
            x = np.zeros(d)  
            R_sq = 0.0  

            for _ in range(N):
                k = r.choice(range(d))  
                z = r.uniform(-delta,delta) 
                x_prop_k = x[k] + z   
                R_sqprop = R_sq - x[k]**2+ x_prop_k**2 
                if R_sqprop < 1.0: 
                    R_sq = R_sqprop
                    x[k]= x_prop_k   
                    pts.append(x)
                    
                    
        Npoints = len(pts)
        Nhits = 0
        # cylinder
        for p in pts:   
            xc = r.uniform(-delta, delta)
            R_sq = xc**2 + sum([g**2 for g in p])
            y = (x, )
            
            if R_sq <= 1: # p and xc are in D_{d+1}
                Nhits += 1
        ratio = Nhits / Npoints 
        # recurrence relation regarding ratii
        D[d + 1] = D[d] * ratio * 2
        
    return D

def mcmc_volume_ergodic_mean(dmax = 10, N = 100, delta = 1.0):
    D = {}
    Z = 0
    N = 100
    samples = 1000 
    D[1] = 2 
    for d in range(1, dmax+1):  
        Nhits = 0
        all = []
        R_sq = 0.0  
        x = np.zeros(d)
        l = 0 
        for _ in range(N * samples):
            k = r.choice(range(d)) if d > 1 else 0 
            z = r.uniform(-delta,delta) 
            x_prop_k = x[k] + z   
            R_sqprop = R_sq - x[k]**2+ x_prop_k**2 ## 
            if R_sqprop < 1.0: 
                    R_sq = R_sqprop
                    x[k]= x_prop_k   
            xc = r.uniform(-delta, delta)
            R_sqcl = R_sq + xc**2 
            if(R_sqcl < 1):
                Nhits += 1
        ratio = Nhits / (N * samples) 
        D[d + 1] = D[d] * ratio * 2
    return D
    


D1 = mcmc_volume(100)
D2 = mcmc_volume_ergodic_mean(100)

print('Volume computation with Ergodic Theorem\n') 
for d in D2.keys():
    Vactual = Vol1(d)
    rel = abs(Vactual - D2[d]) / Vactual
    print('MCMC Estimate for d = {} : {}, Actual = {}, (Abs) Relative Error = {}'.format(d, D2[d], Vactual,  rel))

print('Volume computation with Monte Carlo\n')
for d in D1.keys():
    Vactual = Vol1(d)
    rel = abs(Vactual - D1[d]) / Vactual 
    print('MCMC Estimate for d = {} : {}, Actual = {}, (Abs) Relative Error = {}'.format(d, D1[d], Vactual, rel))
        

Volume computation with Ergodic Theorem

MCMC Estimate for d = 1 : 2, Actual = 1.9999999999999998, (Abs) Relative Error = 1.1102230246251568e-16
MCMC Estimate for d = 2 : 3.1462, Actual = 3.141592653589793, (Abs) Relative Error = 0.0014665639114422135
MCMC Estimate for d = 3 : 4.190864248, Actual = 4.1887902047863905, (Abs) Relative Error = 0.000495141344448119
MCMC Estimate for d = 4 : 4.92485221239472, Actual = 4.934802200544679, (Abs) Relative Error = 0.002016289153162152
MCMC Estimate for d = 5 : 5.286040873651749, Actual = 5.263789013914325, (Abs) Relative Error = 0.0042273464378233035
MCMC Estimate for d = 6 : 5.149343856659115, Actual = 5.167712780049969, (Abs) Relative Error = 0.00355455579144562
MCMC Estimate for d = 7 : 4.685799922682662, Actual = 4.7247659703314016, (Abs) Relative Error = 0.008247191055265394
MCMC Estimate for d = 8 : 4.042814457292147, Actual = 4.058712126416768, (Abs) Relative Error = 0.003916924538980713
MCMC Estimate for d = 9 : 3.309852196185081, Actual